In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import os
import shutil

working_dir = "/kaggle/working"

for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)

    if os.path.isfile(item_path):
        os.remove(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)

print("All contents within /kaggle/working have been deleted.")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pycocotools fiftyone

In [ ]:
import yaml

with open("/kaggle/input/8000-images-dataset/data.yaml") as f:
    data_config = yaml.safe_load(f)
    class_names = data_config['names']  # Get real class names

# Temporary workaround if you actually have 1 class
if len(class_names) == 1:
    class_names.append('background')

In [ ]:
!pip install rfdetr supervision albumentations opencv-python torch torchvision --quiet

In [ ]:
!pip install tqdm
!pip install -U albumentations
!pip install rfdetr torchvision pytorch-lightning
!pip install rfdetr[onnxexport]
!pip install rfdetr supervision albumentations opencv-python torch torchvision --quiet

In [ ]:
import os
import json
import shutil
import yaml
import cv2
import numpy as np
from PIL import Image
from rfdetr import RFDETRBase
from tqdm import tqdm
import torch
import albumentations as A

# =====================
# Configuration
# =====================
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
yolo_dir = "/kaggle/input/8000-images-dataset"
output_dir = "/kaggle/working/coco_dataset"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =====================
# Preprocessing Functions
# =====================
def adjust_brightness_contrast(image, alpha=1.2, beta=15, **kwargs):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def simple_white_balance(image, **kwargs):
    result = image.astype(np.float32)
    avg_b, avg_g, avg_r = np.mean(result, axis=(0,1))
    avg_gray = (avg_b + avg_g + avg_r) / 3
    result[..., 0] *= avg_gray / avg_b
    result[..., 1] *= avg_gray / avg_g
    result[..., 2] *= avg_gray / avg_r
    return np.clip(result, 0, 255).astype(np.uint8)

def histogram_equalization(image, **kwargs):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[..., 0] = cv2.equalizeHist(img_yuv[..., 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

# =====================
# Augmentation Pipeline
# =====================
preprocess = A.Compose([
    A.Lambda(image=lambda image, **kw: adjust_brightness_contrast(image)),
    A.Lambda(image=lambda image, **kw: simple_white_balance(image)),
    A.Lambda(image=lambda image, **kw: histogram_equalization(image)),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
])

# =====================
# Dataset Conversion
# =====================
def process_image(image_path):
    """Apply preprocessing pipeline"""
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    processed = preprocess(image=image)['image']
    return processed

def yolo_to_coco(yolo_root, output_root):
    # Load class names
    with open(os.path.join(yolo_root, "data.yaml")) as f:
        class_names = yaml.safe_load(f)['names']
    
    if len(class_names) == 1:
        class_names.append("background")

    # Create category mapping
    categories = [{"id": i, "name": n, "supercategory": "object"} for i, n in enumerate(class_names)]

    for split in ['train','valid','test']:
        split_dir = os.path.join(output_root, split)
        os.makedirs(split_dir, exist_ok=True)

        coco_data = {
            "info": {"description": "Preprocessed Dataset"},
            "licenses": [{"name": "MIT"}],
            "categories": categories,
            "images": [],
            "annotations": []
        }

        annotation_id = 1
        image_id = 1

        # Process images with preprocessing
        img_dir = os.path.join(yolo_root, split, "images")
        lbl_dir = os.path.join(yolo_root, split, "labels")

        for img_file in tqdm(os.listdir(img_dir)):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            # Process and save image
            src_path = os.path.join(img_dir, img_file)
            dst_path = os.path.join(split_dir, img_file)
            
            # Apply preprocessing chain
            processed_img = process_image(src_path)
            cv2.imwrite(dst_path, cv2.cvtColor(processed_img, cv2.COLOR_RGB2BGR))

            # Get dimensions
            height, width = processed_img.shape[:2]

            # Add image entry
            coco_data["images"].append({
                "id": image_id,
                "file_name": img_file,
                "width": width,
                "height": height,
                "license": 1,
                "date_captured": "2024-01-01"
            })

            # Process labels
            label_path = os.path.join(lbl_dir, os.path.splitext(img_file)[0] + ".txt")
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) != 5: continue

                        cls_id, xc, yc, w, h = map(float, parts)
                        x_min = (xc - w/2) * width
                        y_min = (yc - h/2) * height
                        box_w = w * width
                        box_h = h * height

                        coco_data["annotations"].append({
                            "id": annotation_id,
                            "image_id": image_id,
                            "category_id": int(cls_id),
                            "bbox": [x_min, y_min, box_w, box_h],
                            "area": box_w * box_h,
                            "iscrowd": 0
                        })
                        annotation_id += 1

            image_id += 1

        # Save annotations
        with open(os.path.join(split_dir, "_annotations.coco.json"), 'w') as f:
            json.dump(coco_data, f, indent=2)

# =====================
# Training Setup
# =====================
if __name__ == "__main__":
    # Convert dataset
    import os
    import shutil
    
    working_dir = "/kaggle/working"
    
    for item in os.listdir(working_dir):
        item_path = os.path.join(working_dir, item)
    
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
    
    print("All contents within /kaggle/working have been deleted.")
    yolo_to_coco(yolo_dir, output_dir)

In [ ]:
import torch
import json
import os
import csv
import matplotlib.pyplot as plt
from rfdetr import RFDETRBase  # adjust import as needed

# Define output directory
output_path = "/kaggle/working/output"
os.makedirs(output_path, exist_ok=True)

# Initialize model
model = RFDETRBase(
    resolution=336,
    num_classes=len(class_names),
)

# Freeze the underlying PyTorch model inside RFDETRBase
# Roboflow RFDETRBase wraps the real nn.Module in `model.network`
module_to_freeze = model.network  # this is a torch.nn.Module

# Freeze all parameters
for name, param in module_to_freeze.named_parameters():
    param.requires_grad = False

# Unfreeze only the classification and bbox heads
for head_attr in ['class_embed', 'bbox_embed', 'head']:
    head = getattr(module_to_freeze, head_attr, None)
    if isinstance(head, torch.nn.Module):
        for param in head.parameters():
            param.requires_grad = True

# Optionally print trainable parameters for verification
trainable = [name for name, param in module_to_freeze.named_parameters() if param.requires_grad]
print(f"Trainable parameters: {trainable}")

# Track best model
best_val_acc = 0.0
best_ckpt_path = os.path.join(output_path, "best.pt")

# Metrics history
history = {
    "epoch": [],
    "train_loss": [],
    "val_loss": [],
    "train_acc": [],
    "val_acc": []
}

def on_epoch_end(metrics: dict):
    global best_val_acc

    epoch = len(history["epoch"]) + 1
    train_loss = metrics.get("train_loss", 0.0)
    val_loss = metrics.get("val_loss", 0.0)
    train_acc = metrics.get("train_map50_95", 0.0)
    val_acc = metrics.get("val_map50_95", 0.0)

    # Append to history
    history["epoch"].append(epoch)
    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["train_acc"].append(train_acc)
    history["val_acc"].append(val_acc)

    # Save best checkpoint
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        if os.path.exists(best_ckpt_path):
            os.remove(best_ckpt_path)
        torch.save(model.state_dict(), best_ckpt_path)
        print(f"[Epoch {epoch}] New best model saved to best.pt (val_acc={val_acc:.4f})")

# Register callback
model.callbacks["on_fit_epoch_end"].append(on_epoch_end)

# Prepare trainer (unwrap DataParallel if used)
trainer = model.module if isinstance(model, torch.nn.DataParallel) else model

# Train
trainer.train(
    dataset_dir=output_dir,
    epochs=100,
    batch_size=4,
    grad_accum_steps=4,
    lr=1e-4,
    output_dir=output_path,
    early_stopping=True,
    early_stopping_patience=5,  # updated from 15 to 5
    tensorboard=True,
    wandb=False,
    optimizer_params={"weight_decay": 1e-4},
    use_amp=True
)

print("Training completed successfully!")

# Save metrics to JSON and CSV
with open(os.path.join(output_path, "history.json"), "w") as f:
    json.dump(history, f)

with open(os.path.join(output_path, "history.csv"), "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["epoch", "train_loss", "val_loss", "train_acc", "val_acc"])
    for i in range(len(history["epoch"])):
        writer.writerow([
            history["epoch"][i],
            history["train_loss"][i],
            history["val_loss"][i],
            history["train_acc"][i],
            history["val_acc"][i]
        ])

# Plotting
epochs = history["epoch"]

# Loss plot
plt.figure(figsize=(6, 4))
plt.plot(epochs, history["train_loss"], label="Train Loss")
plt.plot(epochs, history["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs. Epoch")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_path, "loss_vs_epoch.png"))
plt.show()

# Accuracy plot
plt.figure(figsize=(6, 4))
plt.plot(epochs, history["train_acc"], label="Train Accuracy (mAP@.5:.95)")
plt.plot(epochs, history["val_acc"], label="Validation Accuracy (mAP@.5:.95)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Epoch")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_path, "accuracy_vs_epoch.png"))
plt.show()

In [ ]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import shutil
import zipfile
from PIL import Image
from scipy.optimize import linear_sum_assignment
from rfdetr import RFDETRBase
from tqdm import tqdm

# =====================
# Configuration
# =====================
COCO_TEST_DIR = "/kaggle/working/coco_dataset/test"
MODEL_PATH = "/kaggle/input/ema-regular-model-8000-images/checkpoint_best_regular.pth"
IOU_THRESHOLD = 0.5
OUTPUT_DIR = "/kaggle/working/results"
ZIP_PATH = "/kaggle/working/results.zip"
TILE_SIZE = (200, 200)

print("Model exists:", os.path.exists(MODEL_PATH))  # Should return True

# =====================
# Delete existing output
# =====================
for path in [OUTPUT_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isfile(path):
            os.remove(path)
        else:
            shutil.rmtree(path)

# =====================
# COCO Annotation Loader
# =====================
def load_coco_annotations(json_path):
    with open(json_path) as f:
        data = json.load(f)
    
    id_to_image = {img["id"]: img for img in data["images"]}
    annotations = {}
    
    for ann in data["annotations"]:
        image_id = ann["image_id"]
        if image_id not in annotations:
            annotations[image_id] = []
        annotations[image_id].append(ann)
    
    return id_to_image, annotations

# =====================
# IoU Calculation
# =====================
def compute_iou(box1, box2):
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (box1[2]-box1[0]) * (box1[3]-box1[1])
    box2_area = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

# =====================
# Grid Creation Function
# =====================
def create_image_grid(image_paths, grid_path):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, TILE_SIZE)
            images.append(img)
        else:
            images.append(np.zeros((TILE_SIZE[1], TILE_SIZE[0], 3), dtype=np.uint8))
    
    while len(images) < 16:
        images.append(np.zeros((TILE_SIZE[1], TILE_SIZE[0], 3), dtype=np.uint8))
    
    rows = []
    for i in range(0, 16, 4):
        row = np.hstack(images[i:i+4])
        rows.append(row)
    grid = np.vstack(rows)
    cv2.imwrite(grid_path, grid)

# =====================
# Metrics Calculation
# =====================
def calculate_metrics(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

# =====================
# Main Processing
# =====================
def analyze_predictions():
    # Initialize model
    model = RFDETRBase(checkpoint_path=MODEL_PATH)
    
    # Load COCO data
    coco_json = os.path.join(COCO_TEST_DIR, "_annotations.coco.json")
    id_to_image, coco_anns = load_coco_annotations(coco_json)
    
    # Create output directories
    predicted_dir = os.path.join(OUTPUT_DIR, "predicted")
    wrong_dir = os.path.join(OUTPUT_DIR, "wrong")
    os.makedirs(predicted_dir, exist_ok=True)
    os.makedirs(os.path.join(wrong_dir, "iou_threshold"), exist_ok=True)
    os.makedirs(os.path.join(wrong_dir, "label_mismatch"), exist_ok=True)
    
    # Metrics tracking
    global_tp = 0
    global_fp = 0
    global_fn = 0
    failure_data = []
    
    # Process each image
    for image_id in tqdm(id_to_image, desc="Processing Images"):
        img_info = id_to_image[image_id]
        img_path = os.path.join(COCO_TEST_DIR, img_info["file_name"])
        
        # Load image and get predictions
        image = Image.open(img_path)
        detections = model.predict(image)
        pred_boxes = np.array(detections.xyxy)
        
        # Get ground truth boxes
        gt_boxes = []
        if image_id in coco_anns:
            for ann in coco_anns[image_id]:
                x, y, w, h = ann["bbox"]
                gt_boxes.append([x, y, x+w, y+h])
        gt_boxes = np.array(gt_boxes)
        
        # Create annotated image
        processed_img = cv2.imread(img_path)
        filename = os.path.basename(img_path)
        
        # Draw annotations
        if gt_boxes.size > 0:
            for gt in gt_boxes:
                cv2.rectangle(processed_img, 
                            (int(gt[0]), int(gt[1])),
                            (int(gt[2]), int(gt[3])),
                            (0, 255, 0), 2)
        
        if pred_boxes.size > 0:
            for box in pred_boxes:
                cv2.rectangle(processed_img,
                            (int(box[0]), int(box[1])),
                            (int(box[2]), int(box[3])),
                            (0, 0, 255), 2)
        
        # Save to predicted folder
        cv2.imwrite(os.path.join(predicted_dir, filename), processed_img)
        
        # Initialize metrics for current image
        tp = 0
        fp = 0
        fn = 0
        
        has_gt = gt_boxes.size > 0
        has_pred = pred_boxes.size > 0
        
        if has_gt and has_pred:
            cost_matrix = np.zeros((len(gt_boxes), len(pred_boxes)))
            for i, gt in enumerate(gt_boxes):
                for j, pred in enumerate(pred_boxes):
                    cost_matrix[i, j] = -compute_iou(gt, pred)
            row_ind, col_ind = linear_sum_assignment(cost_matrix)
            
            matched_gt = set()
            matched_pred = set()
            
            for i, j in zip(row_ind, col_ind):
                iou_val = compute_iou(gt_boxes[i], pred_boxes[j])
                if iou_val >= IOU_THRESHOLD:
                    tp += 1
                    matched_gt.add(i)
                    matched_pred.add(j)
            
            fp += len(pred_boxes) - len(matched_pred)
            fn += len(gt_boxes) - len(matched_gt)
            
        elif has_pred:
            fp += len(pred_boxes)
        elif has_gt:
            fn += len(gt_boxes)
        
        # Update global metrics
        global_tp += tp
        global_fp += fp
        global_fn += fn
        
        # Failure analysis
        label_mismatch = (len(gt_boxes) != len(pred_boxes)) or (fp > 0) or (fn > 0)
        low_iou = (tp > 0) and (fp + fn > 0)
        
        # Save failure cases
        failure_reasons = []
        if label_mismatch:
            failure_reasons.append("label_mismatch")
            cv2.imwrite(os.path.join(wrong_dir, "label_mismatch", filename), processed_img)
        if low_iou:
            failure_reasons.append("iou_threshold")
            cv2.imwrite(os.path.join(wrong_dir, "iou_threshold", filename), processed_img)
        
        if failure_reasons:
            failure_data.append({
                "image": filename,
                "tp": tp,
                "fp": fp,
                "fn": fn,
                "failure_reasons": ", ".join(failure_reasons)
            })

    # Calculate final metrics
    precision, recall = calculate_metrics(global_tp, global_fp, global_fn)
    
    # Save metrics to file
    metrics_content = f"""Evaluation Metrics:
    - True Positives (TP): {global_tp}
    - False Positives (FP): {global_fp}
    - False Negatives (FN): {global_fn}
    - Precision: {precision:.4f}
    - Recall: {recall:.4f}
    - F1 Score: {2*(precision*recall)/(precision+recall):.4f}"""
    
    with open(os.path.join(OUTPUT_DIR, "metrics.txt"), "w") as f:
        f.write(metrics_content)
    
    # Save CSV report
    df = pd.DataFrame(failure_data)
    csv_path = os.path.join(OUTPUT_DIR, "failure_report.csv")
    df.to_csv(csv_path, index=False)
    
    # Create grids for wrong predictions
    for subfolder in ["iou_threshold", "label_mismatch"]:
        subfolder_path = os.path.join(wrong_dir, subfolder)
        if os.path.exists(subfolder_path):
            image_files = sorted([f for f in os.listdir(subfolder_path) 
                               if f.endswith('.jpg') and not f.startswith('grid_')])
            
            for i in range(0, len(image_files), 16):
                chunk = image_files[i:i+16]
                grid_path = os.path.join(subfolder_path, f"grid_{i//16 + 1}.jpg")
                image_paths = [os.path.join(subfolder_path, f) for f in chunk]
                create_image_grid(image_paths, grid_path)

    # Create zip archive
    shutil.make_archive(ZIP_PATH.rstrip('.zip'), 'zip', OUTPUT_DIR)
    
    print(f"\n✅ Analysis complete!")
    print(f"📦 Download results: {ZIP_PATH}")
    print(metrics_content)

if __name__ == "__main__":
    analyze_predictions()

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [ ]:
zip_dir("/kaggle/working/results",'results-regular.zip')